In [1]:
import torch
import torch.nn as nn

In [4]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [5]:
CONTEXT_SIZE = 2 # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [6]:
# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
    raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out
    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

model = CBOW(vocab_size, EMDEDDING_DIM)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [20]:
word_to_ix

{'about': 0,
 'is': 1,
 'inhabit': 2,
 'pattern': 3,
 'called': 4,
 'programs': 5,
 'idea': 6,
 'processes': 7,
 'they': 8,
 'things': 9,
 'In': 10,
 'we': 11,
 'create': 12,
 'are': 13,
 'The': 14,
 'abstract': 15,
 'effect,': 16,
 'computers.': 17,
 'the': 18,
 'People': 19,
 'that': 20,
 'Computational': 21,
 'with': 22,
 'evolution': 23,
 'manipulate': 24,
 'directed': 25,
 'other': 26,
 'to': 27,
 'by': 28,
 'process': 29,
 'a': 30,
 'As': 31,
 'process.': 32,
 'evolve,': 33,
 'direct': 34,
 'spirits': 35,
 'of': 36,
 'spells.': 37,
 'conjure': 38,
 'data.': 39,
 'computer': 40,
 'rules': 41,
 'beings': 42,
 'our': 43,
 'study': 44,
 'processes.': 45,
 'program.': 46,
 'We': 47,
 'computational': 48}

In [34]:
#TRAINING
for epoch in range(50):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)
        log_probs = model(context_vector)
#         total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]))
        #optimize at the end of each epoch
        optimizer.zero_grad()
#         total_loss.backward()
        optimizer.step()

In [35]:
#TESTING
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

In [36]:
#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: things
